In [ ]:
!pip install peft
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install GPUtil
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=cca41997a53b94cb9640a1e3447e8273500dd98d8661e7932b5ff2c32e9c0377
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [ ]:
from google.colab import userdata
HF_key=userdata.get('HuggingFace_Api_key')

In [ ]:
from datasets import load_dataset
import os
import torch
import GPUtil

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
os.environ['HUGGINGFACE_TOKEN']=HF_key

In [ ]:
squad_dataset=load_dataset('squad')

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
squad_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
base_model="meta-llama/Llama-2-7b-chat-hf"

In [ ]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16

)

In [ ]:
model=AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map='auto',use_auth_token=os.environ['HUGGINGFACE_TOKEN'])

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 42% |


In [ ]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 29.61 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
train_dataset=load_dataset('text', data_files={'train':['/content/Fine-tuning-LLMs/data/hawaii_wf_1.txt','/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt','/content/Fine-tuning-LLMs/data/hawaii_wf_3.txt']}, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 270
})

In [ ]:
train_dataset['text'][13]

'This preliminary after-action report is a critical review detailing our preparedness, initial response, evacuation, coordination with other emergency response agencies, recovery, command and control, identification and notification, media response, wellness of our personnel, and training/equipment. '

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(base_model,use_fast=True,add_eos_token=True,use_auth_token=os.environ['HUGGINGFACE_TOKEN'])
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
tokenized_train_dataset=[]
for phrase in train_dataset['text']:
  tokenized_train_dataset.append(tokenizer(phrase))

In [ ]:
tokenized_train_dataset[1]

{'input_ids': [1, 21122, 29979, 8079, 14861, 3120, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_train_dataset[12]

{'input_ids': [1, 512, 278, 3841, 322, 7378, 1951, 445, 15134, 29892, 278, 22209, 29875, 18923, 10317, 756, 3796, 443, 2388, 456, 5921, 368, 304, 9493, 1749, 2933, 29889, 29871, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model=prepare_model_for_kbit_training(model)

In [ ]:
config=LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

In [ ]:
lora_model=get_peft_model(model,peft_config=config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
from transformers import Trainer,TrainingArguments
import transformers

In [ ]:
train_args=TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    max_steps=100,
    learning_rate=1e-4,
    bf16=False,
    optim="paged_adamw_8bit",
    logging_dir="./log",
    save_strategy="epoch",
    save_steps=50,
    logging_steps=10,
    report_to="none"
)

In [ ]:
trainer=Trainer(
    model=lora_model,
    args=train_args,
    train_dataset=tokenized_train_dataset,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
%time
trainer.train()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.261700
20,2.673300
30,2.233000
40,1.900400
50,1.652200
60,1.259900
70,1.175100
80,0.782800
90,0.717200
100,0.565900


/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1228: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68c41a3f-366ff36003936490392d8972;e56c6281-2cb1-4d88-97da-ce330350ccc2)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-chat-hf.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5

TrainOutput(global_step=100, training_loss=1.6221591901779175, metrics={'train_runtime': 1462.5301, 'train_samples_per_second': 1.094, 'train_steps_per_second': 0.068, 'total_flos': 2030985684811776.0, 'train_loss': 1.6221591901779175, 'epoch': 5.882352941176471})

In [ ]:
from peft import PeftModel

In [ ]:
base_model

'meta-llama/Llama-2-7b-chat-hf'

In [ ]:
ft_model_dir="results/checkpoint-100"

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(base_model,use_fast=True,add_eos_token=True,use_auth_token=os.environ['HUGGINGFACE_TOKEN'])

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
base_mode_load=AutoModelForCausalLM.from_pretrained(base_model,quantization_config=bnb_config,use_auth_token=os.environ['HUGGINGFACE_TOKEN'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model_fine_tuned=PeftModel.from_pretrained(base_mode_load,ft_model_dir)

In [ ]:
# user_question="Maui became the stage for the most tragic natural disaster on which date?"

In [ ]:
user_question="The disaster claimes how many lives and how much is the areas burned"

In [ ]:
prompt=f"Question:{user_question} . Based on the question answer appropriately from the trianed data"

In [ ]:
prompt_tokenized=tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:
# model_fine_tuned.eval()

In [ ]:
with torch.no_grad():
  print(tokenizer.decode(model_fine_tuned.generate(**prompt_tokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question:The disaster claimes how many lives and how much is the areas burned . Based on the question answer appropriately from the trianed data Maui County Police Department (MPD) and other first responder agencies experienced significant challenges with the natural disasters that occurred on August 8, 2023, and thereafter. These challenges included: 1. Communication - The MPD Communications Center was overwhelmed with calls for service and suffered a complete loss of power, rendering the center’s communication systems inoperable. This rendered the department’s ability to dispatch and communicate with responding officers, emergency shelters, and the public inoperable. 2. Staffing - The MPD was already short staffed and faced challenges in maintaining regular staffing levels due to vacant positions. The natural disasters that occurred on August 8, 2023, further depleted the department’s staffing levels as many officers were tasked with evacuating their own families and caring for famil

In [ ]:
def get_folder_size(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.isfile(fp):
                total += os.path.getsize(fp)
    return total

size_bytes = get_folder_size(ft_model_dir)
size_gb = size_bytes / (1024**3)
print(f"Model size: {size_bytes:,} bytes ({size_gb:.2f} GB)")

Model size: 125,285,042 bytes (0.12 GB)


In [ ]:
  lora_model.save_pretrained("./ft-model")

/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1228: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68c429ba-5c87121b323ef5a7549954e9;45d93e04-0af2-4e7c-9e21-4207626e338b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-chat-hf.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
model_final = model_fine_tuned.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
model_final.save_pretrained("./merged-ft-model")

In [ ]:
size_bytes = get_folder_size("./merged-ft-model")
size_gb = size_bytes / (1024**3)
print(f"Model size: {size_bytes:,} bytes ({size_gb:.2f} GB)")

Model size: 3,866,043,434 bytes (3.60 GB)


## Note this for reference

In [ ]:
train_args = TrainingArguments(
    output_dir='./results',              # Where to save checkpoints & logs
    per_device_train_batch_size=4,       # Batch size PER GPU/TPU (not global)
    gradient_accumulation_steps=4,       # Simulate larger batch by accumulating grads
    num_train_epochs=3,                  # Number of times to iterate over dataset
    max_steps=100,                       # Max total steps (overrides epochs if set)
    learning_rate=1e-4,                  # Initial learning rate
    bf16=False,                          # Use bfloat16 precision (better on newer GPUs)
    optim="paged_adamw_8bit",            # Optimizer (paged AdamW, 8-bit memory efficient)
    logging_dir="./log",                 # TensorBoard log dir
    save_strategy="epoch",               # Save checkpoint after each epoch
    save_steps=50,                       # Save every N steps (only used if save_strategy="steps")
    logging_steps=10,                    # Log training metrics every N steps
    report_to="none"                     # Disable logging to services (e.g. WandB, HF Hub)
)
